# Tracing Demo

In [ ]:
# For tracing to work, you first need to link an Application Insights resource to your Azure AI Foundry Project. 
# This is just a basic example of how to do that.
# You can find more information about tracing in the following links:
# https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/trace
# https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/visualize-traces

In [26]:
import logging
import os
import datetime
from azure.identity import AzureCliCredential
from azure.ai.projects import AIProjectClient
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor
from dotenv import load_dotenv
from opentelemetry.trace.status import Status, StatusCode

load_dotenv()
PROJECT_CONNECTION_STRING = os.environ["PROJECT_CONNECTION_STRING"]
logging.getLogger("azure").setLevel(logging.DEBUG)

# --------------------------------------------------------------------
# 1) Load Project Client and Set Up Tracing
# --------------------------------------------------------------------
project_client = AIProjectClient.from_connection_string(
    credential=AzureCliCredential(),
    conn_str=PROJECT_CONNECTION_STRING
)

application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your AI Foundry project page.")
    exit()

configure_azure_monitor(connection_string=application_insights_connection_string)
project_client.telemetry.enable()

# Timestamped scenario name
now = datetime.datetime.now()
timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
tracing_name = f"sample_agents_basics_with_azure_monitor_tracing_{timestamp}"
scenario = os.path.basename(tracing_name)
tracer = trace.get_tracer("my_notebook_tracer")

# --------------------------------------------------------------------
# 2) Helper for Exception Tracking
# --------------------------------------------------------------------
def track_exception(span, exc):
    span.set_status(Status(StatusCode.ERROR, str(exc)))
    span.record_exception(exc)

# --------------------------------------------------------------------
# 3) Run AI Project Pipeline
# --------------------------------------------------------------------
def run_ai_project_pipeline():
    instructions = """
    You are a helpful assistant with the goal to give an overview of the latest relevant AI news
    """
    with tracer.start_as_current_span(scenario) as span:
        try:
            with project_client:
                # Agent creation
                with tracer.start_as_current_span("create_agent") as s:
                    agent = project_client.agents.create_agent(
                        model="gpt-4o",
                        name="my-agent",
                        instructions=instructions
                    )
                    s.set_attribute("agent.id", agent.id)

                # Thread creation
                with tracer.start_as_current_span("create_thread") as s:
                    thread = project_client.agents.create_thread()
                    s.set_attribute("thread.id", thread.id)

                # User message creation
                with tracer.start_as_current_span("create_message") as s1:
                    message_text = "Hello! I'd like to have the latest AI news."
                    message = project_client.agents.create_message(
                        thread_id=thread.id,
                        role="user",
                        content=message_text
                    )
                    s1.set_attribute("input.message_content", message_text)
                    s1.set_attribute("output.message_id", message.id)

                # Run agent
                with tracer.start_as_current_span("create_and_process_run") as s:
                    run = project_client.agents.create_and_process_run(
                        thread_id=thread.id,
                        agent_id=agent.id
                    )
                    s.set_attribute("run.status", run.status)
                    if run.status == "failed":
                        s.set_status(Status(StatusCode.ERROR, "Run failed"))
                        s.set_attribute("run.error", str(run.last_error))
                        logging.error(f"Run failed: {run.last_error}")
                    else:
                        logging.info(f"Run finished with status: {run.status}")

                # List messages
                
                
                messages = project_client.agents.list_messages(thread_id=thread.id)
                assistant_response = messages.get_last_message_by_role("assistant")
                assistant_response_text = assistant_response.content[0].text.value
                logging.info(f"Assistant response: {assistant_response_text}")
                s.set_attribute("output.assistant_response", assistant_response_text)
                return assistant_response

                # Delete agent
                with tracer.start_as_current_span("delete_agent") as s:
                    project_client.agents.delete_agent(agent.id)
                    s.set_attribute("deleted.agent_id", agent.id)
                    logging.info("Deleted agent successfully.")
        except Exception as e:
            track_exception(span, e)
            raise

# --------------------------------------------------------------------
# 4) Local Test Entry Point
# --------------------------------------------------------------------
assistant_response = run_ai_project_pipeline()
assistant_response


{'id': 'msg_4bUgZa08CkwOEdrTGQlckGIc', 'object': 'thread.message', 'created_at': 1744016031, 'assistant_id': 'asst_eHZCO2yQyDDxssMxTCCJxybz', 'thread_id': 'thread_bEMZM0RC6XoZbYg9YmcLprak', 'run_id': 'run_eLTTMtRRIz1pZPLoOKl5LLma', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': "As of October 2023, here is an overview of the latest and most relevant developments in AI:\n\n### 1. **OpenAI Announcements**\n   - OpenAI has integrated new features into ChatGPT, including tools like **DALL·E 3**, enabling users to create images directly through text prompts. Additionally, **multimodal capabilities** (e.g., voice interactions and file analysis) are being rolled out in the ChatGPT app for Pro users.\n   - OpenAI has formed partnerships with businesses, focusing on enterprise applications of AI and introducing ChatGPT for businesses to simplify workflows.\n\n### 2. **Anthropic's Claude AI**\n   - Anthropic, an AI safety-focused company, recently launched **Claude 3**, a co

In [ ]:
# This is a version where each run of the pipeline is tracked in a single separate top-level span.

In [28]:
import logging
import os
import datetime
from azure.identity import AzureCliCredential
from azure.ai.projects import AIProjectClient
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor
from dotenv import load_dotenv
from opentelemetry.trace.status import Status, StatusCode

load_dotenv()
logging.getLogger("azure").setLevel(logging.DEBUG)

# Get timestamped trace name early
now = datetime.datetime.now()
timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
tracing_name = f"sample_agents_basics_with_azure_monitor_tracing_{timestamp}"
scenario = os.path.basename(tracing_name)
tracer = trace.get_tracer("my_notebook_tracer")

def run_ai_project_pipeline():
    with tracer.start_as_current_span(scenario) as span:

        try:
            # Move project client creation *inside* the span
            PROJECT_CONNECTION_STRING = os.environ["PROJECT_CONNECTION_STRING"]
            project_client = AIProjectClient.from_connection_string(
                credential=AzureCliCredential(),
                conn_str=PROJECT_CONNECTION_STRING
            )

            # Enable Azure Monitor tracing
            application_insights_connection_string = project_client.telemetry.get_connection_string()
            if not application_insights_connection_string:
                print("Application Insights was not enabled for this project.")
                exit()
            configure_azure_monitor(connection_string=application_insights_connection_string)
            project_client.telemetry.enable()

            instructions = """
            You are a helpful assistant with the goal to give an overview of the latest relevant AI news
            """
            with project_client:
                with tracer.start_as_current_span("create_agent") as s:
                    agent = project_client.agents.create_agent(
                        model="gpt-4o",
                        name="my-agent",
                        instructions=instructions
                    )
                    s.set_attribute("agent.id", agent.id)

                with tracer.start_as_current_span("create_thread") as s:
                    thread = project_client.agents.create_thread()
                    s.set_attribute("thread.id", thread.id)

                with tracer.start_as_current_span("create_message") as s:
                    msg_text = "Hello! I'd like to have the latest AI news."
                    message = project_client.agents.create_message(
                        thread_id=thread.id,
                        role="user",
                        content=msg_text
                    )
                    s.set_attribute("input.message_content", msg_text)
                    s.set_attribute("output.message_id", message.id)

                with tracer.start_as_current_span("create_and_process_run") as s:
                    run = project_client.agents.create_and_process_run(
                        thread_id=thread.id,
                        agent_id=agent.id
                    )
                    s.set_attribute("run.status", run.status)

                with tracer.start_as_current_span("list_messages") as s:
                    messages = project_client.agents.list_messages(thread_id=thread.id)
                    for msg in messages["data"]:
                        s.add_event(f"{msg['role']}: {msg['content'][0]['text']['value']}")

                with tracer.start_as_current_span("delete_agent") as s:
                    project_client.agents.delete_agent(agent.id)
                    s.set_attribute("deleted.agent_id", agent.id)

        except Exception as e:
            span.set_status(Status(StatusCode.ERROR, str(e)))
            span.record_exception(e)
            raise

run_ai_project_pipeline()
